In [1]:
#importing libraries
import pandas as pd
import numpy as np
import nltk
nltk.download('vader_lexicon')
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import re
# spacy for lemmatization
import spacy
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Toshiba\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

In [3]:
#read the data from twitter
df = pd.read_csv('Tweets.csv')
df = df[['tweets','likes','time']]

In [4]:
def text_cleaner(text,num):
    newString = text.lower()
    newString = re.sub(r'\([^)]*\)', '', newString)
    newString = re.sub(r"[0-9]", "", newString)
    newString = re.sub(',',' ', newString)
    newString = re.sub('-',' ', newString)
    newString = re.sub('_',' ', newString)
    newString = re.sub('&','and', newString)
    newString = re.sub('[()]', '', newString)
    newString = re.sub(';',' ', newString)
    newString = re.sub(r"'s\b","",newString)
    newString = re.sub("[^a-zA-Z]", " ", newString)
    newString = re.sub(r"Ä¢", "", newString)
    newString = re.sub(r"¬∑", "", newString)
    newString = re.sub(r"\'", "", newString)
    newString = re.sub(r"\"", "", newString)
    newString = re.sub(r"\n", "", newString)
    newString = re.sub(r"\r", "", newString)
    return newString.strip().lower()

In [5]:
X = []
for t in df['tweets']:
    X.append(text_cleaner(t,0))

In [6]:
#Soting list in the column back again
df['tweets'] = pd.DataFrame(X)
df['tweets'].head()

0    gemin  protein  complex  advances spurs  spina...
1    mt  irpatnih   nih  nindsnews research led by ...
2    a new care pathway could save babies  lives by...
3    did you know that about   of people diagnosed ...
4    emily holtvluwer  mother of two young children...
Name: tweets, dtype: object

In [7]:
#Tokenise
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(df.tweets))

print(data_words[:4])

[['gemin', 'protein', 'complex', 'advances', 'spurs', 'spinal', 'muscular', 'atrophy', 'discovery', 'naturecomms', 'international', 'collaborators', 'researchers', 'upmc', 'upmcnews', 'stjude', 'genetic', 'cause', 'of', 'neurological', 'disorder', 'by', 'developmental', 'delay', 'ataxia', 'mlo', 'labline', 'https', 'co', 'lafhblugn'], ['mt', 'irpatnih', 'nih', 'nindsnews', 'research', 'led', 'by', 'dr', 'kenneth', 'fiscbheck', 'points', 'to', 'potential', 'new', 'approach', 'for', 'treating', 'spinal', 'muscular', 'atrophy', 'genetic', 'illness', 'that', 'causes', 'neurons', 'responsible', 'for', 'movement', 'to', 'die', 'learn', 'more', 'https', 'co', 'wyunzka', 'sma'], ['new', 'care', 'pathway', 'could', 'save', 'babies', 'lives', 'by', 'improving', 'detection', 'diagnosis', 'and', 'rapid', 'access', 'to', 'life', 'changing', 'treatment', 'for', 'infants', 'born', 'with', 'spinal', 'muscular', 'atrophy', 'everydaycounts', 'https', 'co', 'ibnwwhyne'], ['did', 'you', 'know', 'that', 'a

In [8]:
import re, nltk, spacy, gensim
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out

In [9]:
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# Run in terminal: python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only Noun, Adj, Verb, Adverb
data_lemmatized = lemmatization(data_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:2])

['gemin protein complex advance spur spinal muscular atrophy discovery naturecomms international collaborator researcher upmc upmcnew stjude genetic neurological disorder developmental delay ataxia mlo labline https co lafhblugn', 'mt nih nindsnew research lead fiscbheck point potential new approach treat spinal muscular atrophy genetic illness cause neuron responsible movement die learn more https co wyunzka sma']


In [10]:
# Sklearn
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction import text
from pprint import pprint

In [11]:
add_stop_words = ['https', 'get', 'many', 'have', 'hold', 'sir','need','help','plz','make','available','pls']
stop_words = text.ENGLISH_STOP_WORDS.union(add_stop_words)
vectorizer = CountVectorizer(analyzer='word',
                             token_pattern='[a-zA-Z0-9]{3,}',
                             stop_words=stop_words# num chars > 3
                            )

In [12]:
data_vectorized = vectorizer.fit_transform(data_lemmatized)

In [13]:
word_list = vectorizer.get_feature_names()
count_list = data_vectorized.toarray().sum(axis=0)

In [14]:
print(dict(zip(word_list,count_list)))

{'access': 1, 'actforsma': 1, 'advance': 1, 'affect': 1, 'age': 1, 'alifazal': 1, 'andamp': 2, 'anterior': 1, 'application': 1, 'apply': 1, 'approach': 1, 'ataxia': 1, 'atrophy': 26, 'azpvfsrm': 1, 'baby': 3, 'bachpanamitabh': 1, 'basically': 1, 'battle': 4, 'bear': 2, 'birthday': 1, 'bjqgewhdz': 1, 'bzfnzmop': 1, 'care': 2, 'case': 1, 'castro': 1, 'cause': 1, 'cell': 1, 'change': 1, 'charupragya': 1, 'child': 1, 'chitraaum': 1, 'claim': 1, 'clinician': 1, 'cmfqklte': 1, 'collaborator': 1, 'come': 2, 'complex': 1, 'consider': 1, 'cord': 1, 'cost': 3, 'couldn': 1, 'crore': 5, 'crowd': 1, 'cure': 5, 'curesma': 2, 'cwpoilwmv': 1, 'day': 2, 'dear': 1, 'delay': 1, 'demonstration': 1, 'desperate': 1, 'detection': 1, 'developmental': 1, 'diagnose': 2, 'diagnosis': 2, 'diagnostic': 1, 'diana': 1, 'die': 1, 'different': 1, 'discovery': 1, 'discuss': 1, 'disease': 6, 'disorder': 1, 'dmytro': 1, 'dollar': 1, 'donate': 1, 'donation': 1, 'doner': 1, 'drug': 1, 'dubowitz': 1, 'eckmtit': 1, 'emily': 

In [15]:
# Plotting tools
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline
# Build LDA Model
fixed_seed = 100
lda_model = LatentDirichletAllocation(n_components=3,               # Number of topics
                                      max_iter=10,               # Max learning iterations
                                      learning_method='online',
                                      random_state=np.random.RandomState(fixed_seed),          # Random state
                                      batch_size=128,            # n docs in each learning iter
                                      evaluate_every = -1,       # compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               # Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(data_vectorized)

print(lda_model)  # Model attributes

LatentDirichletAllocation(learning_method='online', n_components=3, n_jobs=-1,
                          random_state=RandomState(MT19937) at 0x1AA320CD990)


In [16]:
# column names
topicnames = ["Topic" + str(i) for i in range(lda_model.n_components)]

# index names
docnames = ["Doc" + str(i) for i in range(len(df.tweets))]

# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic

# Styling
def color_green(val):
    color = 'green' if val > .1 else 'black'
    return 'color: {col}'.format(col=color)

def make_bold(val):
    weight = 700 if val > .1 else 400
    return 'font-weight: {weight}'.format(weight=weight)

# Apply Style
df_document_topics = df_document_topic.head().style.applymap(color_green).applymap(make_bold)
df_document_topics

In [17]:
df_document_topic=df_document_topic.loc[:,['dominant_topic']]
df_document_topic1= df_document_topic.reset_index()

In [18]:
#df_document_topic1.head(10)

In [19]:
df_topic_distribution = df_document_topic1['dominant_topic'].value_counts().reset_index(name="Num Documents")
df_topic_distribution.columns = ['Topic Num', 'Num Documents']
df_topic_distribution

Topic Num  Num Documents
0          2             21
1          1              6
2          0              3

In [20]:
# # Visulaiise the topic keywords
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer, mds='tsne')
panel

PreparedData(topic_coordinates=                 x            y  topics  cluster       Freq
topic                                                      
2     -1015.700562  1338.981201       1        1  66.551419
1      -387.996002  -373.316162       2        1  21.804782
0     -2184.745605   -60.565800       3        1  11.643799, topic_info=          Term      Freq      Total Category  logprob  loglift
224  treatment  3.000000   3.000000  Default  30.0000  30.0000
225       type  2.000000   2.000000  Default  29.0000  29.0000
131       life  3.000000   3.000000  Default  28.0000  28.0000
201        sma  9.000000   9.000000  Default  27.0000  27.0000
178    request  1.000000   1.000000  Default  26.0000  26.0000
..         ...       ...        ...      ...      ...      ...
204     spinal  0.801869  17.775238   Topic3  -4.3281  -0.9482
90       gemin  0.498780   2.541889   Topic3  -4.8029   0.5219
224  treatment  0.502272   3.727824   Topic3  -4.7959   0.1460
132       link  0.496040   2.569005   Topic3  -4.8084   0.5058
53    diagnose  0.492419   1.797173   Topic3  -4.8157   0.8558

[156 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
6         1  0.599276     andamp
6         2  0.599276     andamp
10        2  0.773147   approach
12        1  0.763936    atrophy
12        2  0.176293    atrophy
...     ...       ...        ...
226       3  0.857879       upmc
234       1  0.903358       year
235       2  0.763994      young
236       2  0.747192    youtube
237       1  0.890825  zolgensma

[119 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 1])

In [21]:
def show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=20):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

topic_keywords = show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=20)

df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

Word 0    Word 1   Word 2     Word 3  Word 4   Word 5    Word 6  \
Topic 0   atrophy  muscular     type        sma  spinal  request      know   
Topic 1  muscular   atrophy   spinal  treatment     sma     life     crore   
Topic 2    spinal  muscular  atrophy  fightssma     old    raise  medicine   

            Word 7     Word 8    Word 9 Word 10  Word 11 Word 12  \
Topic 0    protein       free    couldn  regard     upmc  people   
Topic 1  diagnosis        new      cure    save  genetic     die   
Topic 2      share  zolgensma  hospital     sma  rainbow   treat   

               Word 13    Word 14      Word 15     Word 16    Word 17  \
Topic 0  international  treatment  naturecomms  researcher        mlo   
Topic 1    responsible    illness        point   fiscbheck  basically   
Topic 2      hyderabad       head      fbyisqr   tbldsdqgk    disease   

         Word 18    Word 19  
Topic 0  advance  lafhblugn  
Topic 1    treat      cause  
Topic 2   suffer     battle